In [28]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine
import sqlite3

In [29]:
# Create a connection to the SQLite database
con = sqlite3.connect('Titanic.sqlite')

# Read the merged and cleaned csv file created during processing and load it into our SQLite database
pd.read_csv('Resources/Titanic_full_dataset.csv').to_sql('Titanic_Full',con, if_exists='replace')

1306

In [30]:
# Create a SQL query and connect with the database
query = "SELECT * FROM Titanic_Full LIMIT 5"
Titanic_query = pd.read_sql(query,con)

# Display the query as a database
Titanic_query

,index,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,num_related,has_related,has_special_ticket
0,0,0,316,1,3,"Nilsson, Miss. Helmina Josefina",female,26.0,0,0,7.8542,None,S,0,0,0
1,1,1,707,1,2,"Kelly, Mrs. Florence ""Fannie""",female,45.0,0,0,13.5000,None,S,0,0,0
2,2,2,1112,1,2,"Duran y More, Miss. Florentina",female,30.0,1,0,13.8583,None,C,1,1,1
3,3,3,459,1,2,"Toomey, Miss. Ellen",female,50.0,0,0,10.5000,None,S,0,0,1
4,4,4,50,0,3,"Arnold-Franchi, Mrs. Josef (Josefine Franchi)",female,18.0,1,0,17.8000,None,S,1,1,0


In [31]:
# just in case, create a separate table with only the rows where cabin is truthy
cabin_query = "SELECT * FROM Titanic_Full WHERE Cabin IS NOT NULL"
cabin_data = pd.read_sql(cabin_query,con)

# note that the reduced cabin dataset still has >100 rows,
# meeting the threshold for the assignment
print(len(cabin_data))
print(' ')
cabin_data.head(3)

293
 


,index,Unnamed: 0,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,num_related,has_related,has_special_ticket
0,6,6,328,1,2,"Ball, Mrs. (Ada E Hall)",female,36.0,0,0,13.0,D,S,0,0,0
1,8,8,540,1,1,"Frolicher, Miss. Hedwig Margaritha",female,22.0,0,2,49.5,B39,C,2,1,0
2,11,11,338,1,1,"Burns, Miss. Elizabeth Margaret",female,41.0,0,0,134.5,E40,C,0,0,0


In [32]:
# break cabin up into letter and number component
cabin_data['cabin_deck'] = cabin_data['Cabin'].str.extract(r"([A-Za-z])")
cabin_data['cabin_num'] = cabin_data['Cabin'].str.extract(r"([0-9]+)")
cabin_data.drop('Cabin', inplace=True, axis=1)
cabin_data.drop('Name', inplace=True, axis=1)
cabin_data.drop('PassengerId', inplace=True, axis=1)
cabin_data = cabin_data[cabin_data['cabin_num'].notna()]
cabin_data.head(3)

,index,Unnamed: 0,Survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,num_related,has_related,has_special_ticket,cabin_deck,cabin_num
1,8,8,1,1,female,22.0,0,2,49.5,C,2,1,0,B,39
2,11,11,1,1,female,41.0,0,0,134.5,C,0,0,0,E,40
3,19,19,1,1,female,39.0,1,0,55.9,S,1,1,0,E,44


In [33]:
cabin_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 287 entries, 1 to 292
Data columns (total 15 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   index               287 non-null    int64  
 1   Unnamed: 0          287 non-null    int64  
 2   Survived            287 non-null    int64  
 3   Pclass              287 non-null    int64  
 4   Sex                 287 non-null    object 
 5   Age                 287 non-null    float64
 6   SibSp               287 non-null    int64  
 7   Parch               287 non-null    int64  
 8   Fare                287 non-null    float64
 9   Embarked            287 non-null    object 
 10  num_related         287 non-null    int64  
 11  has_related         287 non-null    int64  
 12  has_special_ticket  287 non-null    int64  
 13  cabin_deck          287 non-null    object 
 14  cabin_num           287 non-null    object 
dtypes: float64(2), int64(9), object(4)
memory usage: 35.9+ KB


In [36]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
# create a K Nearest Neighbors Classifier Model
knn = KNeighborsClassifier(n_neighbors=5)

# generic model evaluation method
def ModelEvaluation(model, X_train, y_train, X_test, y_test, verbose=False):
    # fit the model and predict on test and train datasets
    fitted = model.fit(X_train, y_train)
    train_predict = fitted.predict(X_train)
    test_predict = fitted.predict(X_test)

    # allow users to obtain classification report data
    rep1 = classification_report(y_train, train_predict, output_dict=True)
    rep2 = classification_report(y_test, test_predict, output_dict=True)

    if verbose == True:
      # output performance reports
      print("Training Classification Report:")
      print(classification_report(y_train, train_predict))

      print("Testing Classification Report:")
      print(classification_report(y_test, test_predict))

    return rep1, rep2

# this will take a single full titanic-related dataset and
# create scaled train and test sets; a specific subset of columns
# can be specified to reduce the features; an alternate target column
# can be specified for further
def FormatTitanicData(data_frame, dummies, target='Survived', columns = []):
    # if not the default, get a subset of all columns
    if columns != []:
        curr_frame = data_frame[columns]
        # if a column is not in the passed set, do not dummy it
        dummies = [c for c in dummies if c in columns]
    else:
        curr_frame = data_frame

    # split target out from data
    X = curr_frame.drop(target, axis=1)
    y = curr_frame[target]

    # split into train and test sets
    # ! don't change random_state so distribution of 'cabin_deck' does not cause
    # errors due to missing categorie(s); would fix this issue with more time !
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

    # dummy categorical columns
    dummy_train = pd.get_dummies(X_train, columns=dummies)
    dummy_test = pd.get_dummies(X_test, columns=dummies)

    # d_train_cols = dummy_train.columns.to_list()
    # d_test_cols = dummy_test.columns.to_list()
    # if d_train_cols != d_test_cols

    # print(dummy_train.head(), dummy_test.head())

    # scale columns
    scaler = StandardScaler()
    train_scaler = scaler.fit(dummy_train)
    train_scaled = train_scaler.transform(dummy_train)
    test_scaler = scaler.fit(dummy_test)
    test_scaled = test_scaler.transform(dummy_test)

    return train_scaled, y_train, test_scaled, y_test

In [38]:
# check if cabin dataset improves prediction accuracy
dummies = ['Pclass', 'Embarked', 'Sex', 'cabin_deck']
a, b, c, d = FormatTitanicData(cabin_data, dummies)
evals = ModelEvaluation(knn, a, b, c, d, verbose=True)

Training Classification Report:
              precision    recall  f1-score   support

           0       0.78      0.79      0.79        81
           1       0.87      0.87      0.87       134

    accuracy                           0.84       215
   macro avg       0.83      0.83      0.83       215
weighted avg       0.84      0.84      0.84       215

Testing Classification Report:
              precision    recall  f1-score   support

           0       0.70      0.70      0.70        30
           1       0.79      0.79      0.79        42

    accuracy                           0.75        72
   macro avg       0.74      0.74      0.74        72
weighted avg       0.75      0.75      0.75        72



In [39]:
# Close the connection
con.close()